In [ ]:
import sys
import copy
from os import path
import pickle
from dataclasses import dataclass, field

# SOMToolBox_Parse
import pandas as pd
import numpy as np
import gzip

# MetroSolver
import itertools
import logging
from collections import defaultdict

# MiniSOM
import minisom as som
from sklearn import datasets, preprocessing

# SOMViz
from scipy.spatial import distance_matrix, distance
from ipywidgets import Layout, HBox, Box, widgets, interact

import plotly.graph_objects as go
import plotly.express as px
import plotly.colors

logger = logging.getLogger()
logger.handlers.clear()

logger.addHandler(logging.StreamHandler(sys.stdout))

# Set the log level
logger.setLevel(logging.DEBUG)


In [ ]:
class SOMToolBox_Parse:

    def __init__(self, filename):
        self.filename = filename
    
    def read_weight_file(self,):
        df = pd.DataFrame()
        if self.filename[-3:len(self.filename)] == '.gz':
            with gzip.open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)
        else:
            with open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)

        file.close()            
        return df.astype('float64'), vec_dim, xdim, ydim


    def _read_vector_file_to_df(self, df, file):
        xdim, ydim, vec_dim, position = 0, 0, 0, 0
        for byte in file:
            line = byte.decode('UTF-8')
            if line.startswith('$'):
                xdim, ydim, vec_dim = self._parse_vector_file_metadata(line, xdim, ydim, vec_dim)
                if xdim > 0 and ydim > 0 and len(df.columns) == 0:
                    df = pd.DataFrame(index=range(0, ydim * xdim), columns=range(0, vec_dim))
            else:
                if len(df.columns) == 0 or vec_dim == 0:
                    raise ValueError('Weight file has no correct Dimensional information.')
                position = self._parse_weight_file_data(line, position, vec_dim, df)
        return df, vec_dim, xdim, ydim


    def _parse_weight_file_data(self, line, position, vec_dim, df):
        splitted=line.split(' ')
        try:
            df.values[position] = list(np.array(splitted[0:vec_dim]).astype(float))
            position += 1
        except: raise ValueError('The input-vector file does not match its unit-dimension.') 
        return  position


    def _parse_vector_file_metadata(self, line, xdim, ydim, vec_dim):
        splitted = line.split(' ')
        if splitted[0] == '$XDIM':      xdim = int(splitted[1])
        elif splitted[0] == '$YDIM':    ydim = int(splitted[1])
        elif splitted[0] == '$VEC_DIM': vec_dim = int(splitted[1])
        return xdim, ydim, vec_dim 
        

In [ ]:
class MetroSolver:
    """ Metro Map Solver
    
    """
    def __init__(self, lines, input_grid, metro_grid=None, corner_penalties=[0.0, 0.7, 1.4, 4.2, 5.6]):
        self.__lines = copy.deepcopy(lines)
        if metro_grid is None:
            metro_grid = input_grid
        self.__input_grid = input_grid
        self.__metro_grid = metro_grid
        self.__scale = np.array([mg / ig for ig, mg in zip(self.__input_grid, self.__metro_grid)])

        self.__raw_solution = None
        self.__solution = None
        
        if not isinstance(corner_penalties, list):
            raise TypeError('`penalties` has to be a list of numbers')
        if not np.all([isinstance(i, float) or isinstance(i, int) for i in corner_penalties]):
            raise TypeError('`penalties` has to be a list of numbers')
        if not len(corner_penalties) == 5:
            raise ValueError('`penalties` has to have exactly 5 elements')
        self.__corner_penalties = corner_penalties


    def __transform(self, input_data):
        """ Transform into metro grid coordinates
        
        Utility function to transform a list of lines from input grid coordinates into 
        metro grid coordinates.
        """
        if isinstance(input_data, list):
            return [self.__scale * line for line in input_data]
        elif isinstance(input_data, np.ndarray):
            return self.__scale * input_data
        else:
            raise ValueError(f'Transforming the coordinates contained in a {type(input_data)}'
                             ' is currently not supported.')


    def __inverse_transform(self, input_data):
        """ Transform into input grid coordinates
        
        Utility function to transform a list of lines from metro grid coordinates into 
        input grid coordinates.
        """
        if isinstance(input_data, list):
            return [line / self.__scale for line in input_data]
        elif isinstance(input_data, np.ndarray):
            return input_data / self.__scale
        else:
            raise ValueError(f'Transforming the coordinates contained in a {type(input_data)}'
                             ' is currently not supported.')


    @staticmethod
    def __neighborhood_generator():
        """Sequentially point generator
        
        Here we generate 2D points at discrete positions starting from the origin.
        The generated points are spiraling outwards towards infinity.
        """
        r = 1
        pos = np.array([0,0])
        directions = [np.array([1,0]), np.array([0,-1]), np.array([-1,0]), np.array([0,1])]
        d_idx = 0
        yield pos
        pos = np.array([0,r])
        yield pos
        while True:
            if np.abs(pos[0]) == np.abs(pos[1]): # corner
                if d_idx == 3: # completed 4th direction
                    r += 1
                    d_idx = 0
                    pos[1] = r
                    yield pos
                    continue
                d_idx += 1
            pos = pos + directions[d_idx]
            yield pos


    def __gen_neighbors(self, pt, n_neighbors = 5):
        neighbors = []
        nearest = np.round(pt)

        for offset in self.__neighborhood_generator():
            new_pt = nearest + offset

            neighbors.append((new_pt, np.linalg.norm(new_pt-pt)))
            if len(neighbors) >= n_neighbors:
                break

        neighbors.sort(key=lambda a: a[1])
        return neighbors


    @staticmethod
    def __get_sector(v1, v2):
        """ Returns the octilinear sector of v2 relative to v1
        
        Determines the octilinear sector of v2 relative to v1. The code is based
        on a similar calculation in the MetroMapVisualizer.java which is part of
        the Java SOMToolbox (http://www.ifs.tuwien.ac.at/dm/somtoolbox/index.html)
        
        The numbering of the different sectors is as follows (v1 in the center):
            3  2  1
            4 -1  0
            5  6  7
        """

        if v1[0] < v2[0] and v1[1] == v2[1]:   # right
            return 0
        elif v1[0] < v2[0] and v1[1] < v2[1]:  # right up
            return 1
        elif v1[0] == v2[0] and v1[1] < v2[1]: # up
            return 2
        elif v1[0] > v2[0] and v1[1] < v2[1]:  # left up
            return 3
        elif v1[0] > v2[0] and v1[1] == v2[1]: # left
            return 4
        elif v1[0] > v2[0] and v1[1] > v2[1]:  # left down
            return 5
        elif v1[0] == v2[0] and v1[1] > v2[1]: # down
            return 6
        elif v1[0] < v2[0] and v1[1] > v2[1]:  # right down
            return 7
        if np.any(v1 != v2):
            print(v1, v2)

        # points equal
        return -1 


    def __calc_penalty(self, sector_diff):
        """ Calculate a penalty for line bends
        
        Calculates a penalty based on the sector difference of two edges.
        A penalty of 3 makes the specific configuration as undesirable as a node
        that's 3 cells apart. Closer solutions are favored in the final selection.
        """
        if sector_diff == 4:
            return self.__corner_penalties[4]

        sector_diff = np.mod(sector_diff, 4)
        if sector_diff == 0:
            return self.__corner_penalties[0]
        elif sector_diff == 1:
            return self.__corner_penalties[1]
        elif sector_diff == 2:
            return self.__corner_penalties[2]
        elif sector_diff == 3:
            return self.__corner_penalties[3]


    def __gen_feasible_neighbors(self, snapped, pt, n_neighbors = 4):
        # search more neighbors than return, increases quality of neighbors with a 
        # relatively small additional computational effort
        MULTIPLIER = 2

        neighbors = []
        nearest = np.round(pt)
        prev = snapped[-1]

        prev_direction = None
        prev_sector = None
        if len(snapped) > 1:
            prev_sector = self.__get_sector(snapped[-2], snapped[-1])

        for offset in self.__neighborhood_generator():
            new_pt = nearest + offset

            diff = np.abs(new_pt - prev)
            if new_pt[0] == prev[0] or new_pt[1] == prev[1] or diff[0] == diff[1]:
                penalty = 0
                if prev_sector is not None:
                    new_sector = self.__get_sector(prev, new_pt)
                    abs_sec_diff = np.abs(new_sector - prev_sector)
                    penalty = self.__calc_penalty(abs_sec_diff)

                dist = np.linalg.norm(new_pt - prev)
                if dist >= 1:
                    neighbors.append((new_pt, np.linalg.norm(new_pt - pt) + penalty))

            if len(neighbors) >= n_neighbors * MULTIPLIER:
                break

        neighbors.sort(key=lambda a: a[1])
        return neighbors[:n_neighbors]


    def __snap_line(self, line, snapped, dist, lb):
        if len(line) == 0:
            return dist, snapped
        pt = line[0]
        best_line = None
        best = None
        neighbors = self.__gen_feasible_neighbors(snapped, pt)
        for n_pt, n_dist in neighbors:
            if (dist + n_dist) > lb:
                continue
            n_snapped = snapped[:]
            n_snapped.append(n_pt)
            total_dist, new_snapped = self.__snap_line(line[1:], n_snapped, dist + n_dist, lb)
            if total_dist is not None and total_dist < lb:
                lb = total_dist
                best = total_dist
                best_line = new_snapped
        return best, best_line


    def solve(self):
        dist_threshold = 3
        snapped_lines = []
        lines = self.__transform(self.__lines)
        for idx, line in enumerate(lines):
            logging.info(f"snapping line {idx+1}/{len(self.__lines)}")
            start = line[0]
            neighbors = self.__gen_neighbors(start)
            best_dist = 999999
            snapped = None
            for pt, dist in neighbors:
                n_dist, n_snapped = self.__snap_line(line[1:], [pt], dist, best_dist)
                if n_dist is None:
                    continue
                if n_dist < best_dist:
                    #print(n_snapped)
                    best_dist = n_dist
                    snapped = n_snapped
            
            snapped_lines.append(snapped)

        self.__raw_solution = snapped_lines


    def post_process(self, overlap_shift):
        """ Post processing
        
        Move overlapping metro lines so that each individual line is visible in the final
        plot.
        
        TODO: Return a list of crossover stations.
        """
        
        if self.__raw_solution is None:
            print('The solve method has to be called before any post processing can be done.')
            return
        
        lines = copy.deepcopy(self.__raw_solution)
        stations = defaultdict(lambda: np.array([0,0,0,0]))

        # These vectors specify the octilinear base directions. Their index corresponds
        # to the value of the sector.
        orientation_vectors = [
            np.array([1,0]),
            np.array([1,1]),
            np.array([0,1]),
            np.array([-1,1]),
            np.array([-1,0]),
            np.array([-1,-1]),
            np.array([0,-1]),
            np.array([1, -1])
        ]
        
        # These vectors are the normalized orthogonal directions to the first four octilinear
        # directions.
        inv_sqrt = 1/np.sqrt(2)
        shift_vectors = [
            np.array([0,1]),
            np.sqrt(2) * np.array([0,1]),
            np.array([-1,0]),
            np.sqrt(2) * np.array([0,-1])
        ]
        
        grid = defaultdict(lambda: [])
        rank = {}
        
        for lidx, line in enumerate(lines):
            for sidx, (a,b) in enumerate(zip(line[:-1], line[1:])):
                orientation = self.__get_sector(a, b)
                current = np.copy(a)
                vector = orientation_vectors[orientation]
                logger.debug(f"curr: {current} vector: {vector} b {b} orientation {orientation}")
                
                # First iterate over each grid point between the station a and the next station b
                # and find the maximum number of collinear edges (edges that run in the same
                # direction or in reverse) throughout the path between the two stations.
                max_collinear = 0
                while not np.all(np.isclose(current, b, rtol = 0.01)):
                    n_collinear_edges = len(grid[(tuple(current), orientation % 4)])
                    if n_collinear_edges > max_collinear:
                        max_collinear = n_collinear_edges
                    
                    # Move to the next grid point
                    current += vector

                logger.debug(f'max shift {max_collinear}')
                
                # Store the maximum number of collinear lines that are associated to this particular
                # metro line (identified by the line index lidx) and edge (identified by the first
                # stop of that particular edge) of that metro line, as well as its orientation.
                if (lidx, sidx, orientation % 4) in rank:
                    i = sidx
                    while (lidx, i, orientation % 4) in rank:
                        if rank[(lidx, i, orientation % 4)] < max_collinear + 1:
                            rank[(lidx, i, orientation % 4)] = max_collinear + 1
                        i -= 1
                else:
                    rank[(lidx, sidx, orientation % 4)] = max_collinear + 1
                
                # Also update the next station's rank
                rank[(lidx, sidx+1, orientation % 4)] = max_collinear + 1

                # Then iterate over those grid points between the two consecutive stations again
                # and this time assign the maximum number of collinear edges we obtained during the 
                # last step to the actual grid positions.
                current = np.copy(a)
                while not np.all(np.isclose(current, b, rtol = 0.01)):
                    grid[(tuple(current), orientation % 4)].append(True)
                    current += vector
                    
                # stations
                transformed_pt = self.__inverse_transform(a)
                stations[tuple(transformed_pt)][orientation % 4] = stations[tuple(transformed_pt)][orientation % 4] + 1
                if sidx == len(line) - 2:
                    transformed_pt = self.__inverse_transform(b)
                    stations[tuple(transformed_pt)][orientation % 4] = stations[tuple(transformed_pt)][orientation % 4] + 1

        for (lidx, s, orientation), n in rank.items():
            if n > 1:
                f = (n % 2) * 2 - 1
                shift = shift_vectors[orientation] * f * int(n/2) * overlap_shift
                lines[lidx][s] = lines[lidx][s] + self.__transform(shift)
                

        self.__solution = lines
        self.stations = stations
    
    
    def get_raw_solution(self):
        if self.__raw_solution is not None:
            return self.__inverse_transform(self.__raw_solution)
        else:
            logger.warning('Solver has yet to be run. No solution available.')


    def get_solution(self):
        if self.__solution is not None:
            return self.__inverse_transform(self.__solution)
        else:
            logger.warning('Postprocessing has yet to be run. No solution available.')
                

In [ ]:
class SomViz:
    
    def __init__(self, weights=[], m=None, n=None):
        self.weights = weights
        self.m = m
        self.n = n
        
        # Params for the metro visualization
        self.solver_params = None
        self.postprocessing_params = None
        self.lines = None
        self.metro_lines = None
        self.stops = None
        self.solver = None
        self.metro_grid = None
        self.corner_penalties = None


    def umatrix(self, som_map=None, color="Viridis", interp = "best", title=""):
        um =np.zeros((self.m *self.n, 1))
        neuron_locs = list()
        for i in range(self.m):
            for j in range(self.n):
                neuron_locs.append(np.array([i, j]))
        neuron_distmat = distance_matrix(neuron_locs,neuron_locs)

        for i in range(self.m * self.n):
            neighbor_idxs = neuron_distmat[i] <= 1
            neighbor_weights = self.weights[neighbor_idxs]
            um[i] = distance_matrix(np.expand_dims(self.weights[i], 0), neighbor_weights).mean()

        if som_map is None:
            return self.plot(um.reshape(self.m,self.n), color=color, interp=interp, title=title)    
        else:
            som_map.data[0].z = um.reshape(self.m,self.n)


    def hithist(self, som_map=None, idata = [], color='RdBu', interp = "best", title=""):
        hist = [0] *self.n *self.m
        for v in idata: 
            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            hist[position] += 1    
        
        if som_map is None:
            return self.plot(np.array(hist).reshape(self.m,self.n), color=color, interp=interp, title=title)        
        else:
            som_map.data[0].z = np.array(hist).reshape(self.m,self.n)


    def component_plane(self, som_map=None, component=0, color="Viridis", interp = "best", title=""):
        if som_map is None:
            return self.plot(self.weights[:,component].reshape(-1,self.n), color=color, interp=interp, title=title)   
        else:
            som_map.data[0].z = self.weights[:,component].reshape(-1,n)


    def __gen_sequential_colors(self, levels, colors=px.colors.sequential.Jet):
        """Generate a color sequence
        
        Generates a color sequence with the specified number of levels based on the
        provided continuous colormap.
        """
        color_sequence = []
        n_colors = len(colors)
        n_levels = levels

        color_sequence.append(colors[0])

        if n_colors > 1:
            color_step = 1 / (n_colors - 1)
        else:
            return color_sequence * n_levels
    
        for i in range(1, n_levels-1):
            level_pos = i / (n_levels - 1)
            color_index = int(level_pos/color_step)

            intermediate = (level_pos - color_index * color_step)/color_step
            color_sequence.append(plotly.colors.find_intermediate_color(colors[color_index], colors[color_index+1], intermediate, colortype='rgb'))

        color_sequence.append(colors[-1])
        return color_sequence

    
    @staticmethod
    def generate_crossover_station_shape(pos=np.array([0,0]), scale=1, lines_per_sector=np.array([1,0,0,0])):
        leading_sectors = np.argwhere(lines_per_sector == np.max(lines_per_sector)).flatten()
        print(f"leading: {leading_sectors}")
        
        sector = 0
        if len(leading_sectors) == 1:
            sector = (leading_sectors[0] + 2) % 4
        elif len(leading_sectors) == 2:
            secdiff = np.abs(leading_sectors[0] - leading_sectors[1]) % 3
            print(f"secdiff: {secdiff}")
            if secdiff == 0:
                sector = (leading_sectors[0] + 2) % 4
            elif secdiff == 1:
                sector = np.where(lines_per_sector != lines_per_sector[leading_sectors[0]])[0][0]
            elif secdiff == 2:
                sector = (leading_sectors[0] + 1) % 4
        if len(leading_sectors) == 3:
            sector = np.argmin(lines_per_sector)
            
        print(f'lps: {lines_per_sector}')
        print(f"sector: {sector}")

        w = 1
        h = .5
        r = np.min([0.3, h/2, w/2])

        p = np.array([
            [-w/2+r, -h/2  ],
            [w/2-r,  -h/2  ],
            [w/2,    -h/2  ],
            [w/2,    -h/2+r],
            [w/2,    h/2-r ],
            [w/2,    h/2   ],
            [w/2-r,  h/2   ],
            [-w/2+r, h/2   ],
            [-w/2,   h/2   ],
            [-w/2,   h/2-r ],
            [-w/2,   -h/2+r],
            [-w/2,   -h/2  ],
            [-w/2+r, -h/2  ],
        ])

        sector_to_degree = {
            0: -90,
            1: 45,
            2: 0,
            3: -45,
        }

        phi = np.radians(sector_to_degree[sector % 4])

        A = np.array([
            [np.cos(phi), -np.sin(phi)],
            [np.sin(phi), np.cos(phi)],
        ])

        p = pos + scale*(A@(p.T)).T

        return f"""
                M {p[0,0]} {p[0,1]}
                L {p[1,0]} {p[1,1]}
                Q {p[2,0]} {p[2,1]} {p[3,0]} {p[3,1]}
                L {p[4,0]} {p[4,1]}
                Q {p[5,0]} {p[5,1]} {p[6,0]} {p[6,1]}
                L {p[7,0]} {p[7,1]}
                Q {p[8,0]} {p[8,1]} {p[9,0]} {p[9,1]}
                L {p[10,0]} {p[10,1]}
                Q {p[11,0]} {p[11,1]} {p[0,0]} {p[0,1]}
                Z"""


    def __prepare_metro(self, stops, metro_grid, corner_penalties, postprocess, overlap_shift):
        if self.lines is None or stops != self.stops:
            self.lines = []
            n_lines = self.weights.shape[1]
            for component in range(n_lines):
                # Reshape the weights into a 2D array with the dimension (m, n)
                raw = self.weights[:,component].reshape(self.m, self.n)
                # Create a list of values that uniformly divide the range between the
                # minimum and the maximum value of the component weights into 'stops'
                # intervals.
                ranges = np.linspace(raw.min(), raw.max(), stops)
                # Digitize the weights array, so that only 'stops' different values
                # are possible. The value of each element in the resulting grid
                # corresponds to the index of the interval, starting with 1.
                binned = np.digitize(raw, ranges)
                
                stations = []
                for i in range(1, stops+1):
                    # Find the positions inside the digitized array, where the elements
                    # have the value of the current level i
                    match = np.argwhere(binned == i)
                    if match.shape[0] == 0:
                        logging.info("layer empty")
                        continue
                    # Summing up all matches from above, which are provided as an array of
                    # coordinates, and dividing them by the number of matches gives us the
                    # center of gravity for that particular level.
                    stations.append(np.sum(match, axis=0)/match.shape[0])
                self.lines.append(stations)

        solver_activated = False
        if stops != self.stops or metro_grid != self.metro_grid or corner_penalties != self.corner_penalties:
            logger.info('Solving...')

            self.solver = MetroSolver(lines=self.lines, input_grid=(self.m, self.n), metro_grid=metro_grid)
            self.solver.solve()
            solver_activated = True

        if postprocess == True and (solver_activated == True or overlap_shift != self.overlap_shift):
            logger.info('Postprocessing ...')
            self.solver.post_process(overlap_shift=overlap_shift)

        if postprocess:
            self.metro_lines = self.solver.get_solution()
        else:
            self.metro_lines = self.solver.get_raw_solution()

        self.stops = stops
        self.metro_grid = metro_grid
        self.overlap_shift = overlap_shift
        self.corner_penalties = corner_penalties


    def metro(self, som_map=None, stops=6, water_level=0.33, snapping_overlay=True, metro_grid=(10,10), corner_penalties=None, postprocess=True, overlap_shift=0.5):
        water_level = np.clip(water_level, 0, 1)
        water = [
            (0.0, 'rgb(255,255,255)'),
            (water_level, 'rgb(255,255,255)'),
            (water_level, 'rgb(198,219,239)'),
            (1.0, 'rgb(198,219,239)')
        ]

        self.__prepare_metro(stops, metro_grid, corner_penalties, postprocess, overlap_shift)

        if som_map is None:
            som_map = self.umatrix(color=water, interp='best', title='U-matrix SOMToolBox') 

        colors = self.__gen_sequential_colors(len(self.lines), colors=px.colors.diverging.Portland)
        
        if not snapping_overlay:
            for i, (line, col) in enumerate(zip(self.lines, colors)):
                y, x = list(zip(*line))
                som_map.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name=f'Component {i}', line_shape='linear', line=dict(dash='dot', color=col)))
        else:
            for i, (line, col) in enumerate(zip(self.metro_lines, colors)):
                y, x = list(zip(*line))
                som_map.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name=f'Component {i}', line_shape='linear', line=dict(width=6, color=col), marker=dict(size=5, color='white', line=dict(width=2, color='black'))))
                # Add special markers to the endstop corresponding to the highest component value
                som_map.add_trace(go.Scatter(x=[x[0]], y=[y[0]], mode='markers', marker_symbol='circle-x', showlegend=False, line=dict(width=6, color=col), marker=dict(size=10, line=dict(width=2,color='black'))))


            shapes = []
            for (y, x), station_sizes in self.solver.stations.items():
                abs_size = np.sum(station_sizes)

                if abs_size > 1:
                    shapes.append(
                        dict(
                            type="path",
                            path=self.generate_crossover_station_shape(pos=np.array([x, y]),
                                                                       scale=np.array([self.m/16, self.n/16]),
                                                                       lines_per_sector=station_sizes),
                            line_color="black",
                            fillcolor="white",
                        )
                    )


            if shapes:
                som_map.update_layout(shapes=shapes)
        return som_map


    def plot(self, matrix, color="Viridis",interp = "none", title="", showscale=False):
        return go.FigureWidget(go.Heatmap(z=matrix, zsmooth=interp, showscale=showscale, colorscale=color), layout=go.Layout(width=700, height=700,title=title, title_x=0.5, plot_bgcolor='rgb(255,255,255)'))


## MiniSOM

In [ ]:
# Visualizaton
m = 40
n = 20

# Pre-Process dataset
iris = datasets.load_iris().data
min_max_scaler = preprocessing.MinMaxScaler()
iris = min_max_scaler.fit_transform(iris)

# Train SOM
s = som.MiniSom(m, n, iris.shape[1], sigma=0.8, learning_rate=0.7)
s.train_random(iris, 10000, verbose=False)

In [ ]:
s = pickle.load(open(path.join('pretrained', 'overlapping_som.p'), 'rb'))
m = 20
n = 20

In [ ]:
# Visualizaton
viz = SomViz(s._weights.reshape(-1,4), m, n)

In [ ]:
display(
    viz.metro(
        stops=7,
        water_level=0.2,
        metro_grid=(10, 10),
        overlap_shift=0.2,
    )
)

## Read from SOMToolbox

In [ ]:
trainedmap = SOMToolBox_Parse(path.join('pretrained', 'iris', 'iris.vec'))
idata, idim, idata_x, idata_y = trainedmap.read_weight_file()

smap = SOMToolBox_Parse(path.join('pretrained', 'iris', 'iris.wgt.gz'))
smap, sdim, smap_x, smap_y = smap.read_weight_file()


# Visualizaton
viz_iris = SomViz(smap.values.reshape(-1,sdim), smap_y, smap_x)

In [ ]:
display(
    viz_iris.metro(
        stops=7,
        water_level=0.3,
        metro_grid=(10, 10),
        overlap_shift=0.15,
    )
)

In [ ]:
trainedmap = SOMToolBox_Parse(path.join('pretrained', '10clusters', '10clusters.vec'))
idata, idim, idata_x, idata_y = trainedmap.read_weight_file()

smap = SOMToolBox_Parse(path.join('pretrained', '10clusters', '10clusters.wgt'))
smap, sdim, smap_x, smap_y = smap.read_weight_file()

# Visualizaton
viz_10clusters = SomViz(smap.values.reshape(-1,sdim), smap_y, smap_x)

In [ ]:
display(
    viz_10clusters.metro(
        stops=7,
        water_level=0.3,
        metro_grid=(10, 10),
        overlap_shift=0.2,
        postprocess=True,
    )
)

In [ ]:
trainedmap = SOMToolBox_Parse(path.join('pretrained', 'chainlink', 'chainlink.vec'))
idata, idim, idata_x, idata_y = trainedmap.read_weight_file()

In [ ]:
idata

In [ ]:
# Visualizaton
m = 40
n = 20

# Pre-Process dataset
min_max_scaler = preprocessing.MinMaxScaler()
dat = min_max_scaler.fit_transform(idata)

# Train SOM
s = som.MiniSom(m, n, idata.shape[1], sigma=0.8, learning_rate=0.7)
s.train_random(dat, 1000000, verbose=False)

In [ ]:
# Visualizaton
viz_chainlink = SomViz(s._weights.reshape(-1,3), 40, 20)
display(viz_chainlink.umatrix())
display(viz_chainlink.metro(stops=10))